In [1]:
import importlib
import utils

# Reload utils module
importlib.reload(utils)

# import 
from utils import *

## Load Float Model

In [2]:
model = ResNet18().to(device)
model.load_state_dict(torch.load("resnet18_float32.pth"))

<All keys matched successfully>

### AdaRound (learnable rounding per weight)

In [ ]:
adaround_params = []
for name, p in model.named_parameters():
    if 'weight' in name and p.dim() > 1:
        scale = weight_scales[name]
        beta = torch.zeros_like(p.data)
        beta = nn.Parameter(beta)
        adaround_params.append(beta)
        # register beta to param (custom way)
        p.adaround_beta = beta

optimizer = optim.SGD(adaround_params, lr=4e-3)

### 200 step calibration with reconstruction + task losss

In [ ]:
model.train() # for grad
for step in range(200):
    optimizer.zero_grad()
    loss_rec = 0
    for name, 